# Testing quantile function

This code tests my quntile function that I needed to write to deal with the fact I can't download Numpy 2.0 because of various conflicts. 

In [1]:
import numpy as np 
import scipy.stats

In [10]:
# Define a reasonable mean vector and covariance matrix
mean = [10, 1, 0.3, 0.02, 0.01, 0.005]  # Expanded mean vector

covariance = np.array([
    [1, 0.5, 0.2, 0.1, 0.3, 0.2],
    [0.5, 1, 0.3, 0.2, 0.4, 0.3],
    [0.2, 0.3, 1, 0.4, 0.5, 0.4],
    [0.1, 0.2, 0.4, 1, 0.6, 0.5],
    [0.3, 0.4, 0.5, 0.6, 1, 0.7],
    [0.2, 0.3, 0.4, 0.5, 0.7, 1]
])  # Expanded covariance matrix

In [11]:
samples_main = scipy.stats.multivariate_normal(
    mean, covariance, allow_singular=True
).rvs(size=10000)

In [12]:
num_amplitude_params = 4
samples_re = samples_main[:, :num_amplitude_params:2]  
samples_im = samples_main[:, 1:num_amplitude_params:2]  
sample_abs_amplitudes = np.sqrt(samples_re**2 + samples_im**2) 

In [13]:
log_samples = np.log(sample_abs_amplitudes)
samples_weights = np.exp(-np.sum(log_samples, axis=1))

In [14]:
def weighted_quantile(values, quantiles, weights=None):
    values = np.array(values)
    quantiles = np.array(quantiles)
    if weights is None:
        weights = np.ones(values.shape[0])
    weights = np.array(weights)

    # Sort values and weights along the first axis
    sorter = np.argsort(values, axis=0)
    sorted_values = np.take_along_axis(values, sorter, axis=0)
    sorted_weights = np.take_along_axis(weights[:, None], sorter, axis=0)

    # Compute cumulative weights
    cumulative_weights = np.cumsum(sorted_weights, axis=0) - 0.5 * sorted_weights
    cumulative_weights /= cumulative_weights[-1, :]
    
    # Interpolate quantiles
    quantile_values = np.empty((len(quantiles), values.shape[1]))
    for i in range(values.shape[1]):
        quantile_values[:, i] = np.interp(quantiles, cumulative_weights[:, i], sorted_values[:, i])

    return quantile_values

In [15]:
quantiles = [0.1, 0.25, 0.5, 0.75, 0.9]
quantile_vals = weighted_quantile(sample_abs_amplitudes, quantiles, weights=samples_weights)
quantile_vals_test = np.quantile(sample_abs_amplitudes, quantiles, method='inverted_cdf', weights=samples_weights, axis=0)

In [16]:
print(quantile_vals)

[[ 8.63208146  0.09224918]
 [ 9.22636853  0.27377728]
 [ 9.89135792  0.63616344]
 [10.62027898  1.10322726]
 [11.25475623  1.62714265]]


In [17]:
print(quantile_vals_test)

[[ 8.63211544  0.09226557]
 [ 9.22684304  0.27381485]
 [ 9.89136437  0.63612787]
 [10.62028231  1.10322339]
 [11.25488889  1.62717964]]
